# Explanation of this notebook: (09/27/2025)

- This is essentially a copy of the 20250909_validate_jpar_calc.ipynb notebook basically line for line, with the exception of two parts (I think):
    - Added some of the relevant columns from both 20250209_joined_events_members.csv and Cumulative JPAR Event Data - Event Information.csv Google sheet
    - Copied calculation_df after the JPAR calculation to reformat some columns to be compatible with existing code of mine.

- There are a few things I need to clean up (e.g., NA's I've ignored, among other issues, but will clean that up later!). Just wanted a working demo.
- The output of this is the input to the dashboard.

In [1]:
# Import required packages
import pandas as pd
import numpy as np
import re

import os


from datetime import datetime

In [2]:
# Read in the data
joined_events_members_filtered = pd.read_csv('../data_jpar/20250209_joined_events_members.csv')
jpar_times = pd.read_csv('../data_jpar/jpar_times.csv')

# remove ppl who joined after the cuttoff
joined_events_members_filtered = joined_events_members_filtered[
    pd.to_datetime(joined_events_members_filtered['initiated_date'], errors='coerce') < pd.Timestamp('2025-02-09')
]

# and exclude connor
# Drop row where full_name == 'conner delaat'
joined_events_members_filtered = joined_events_members_filtered[joined_events_members_filtered['full_name']!= 'conner delaat']


FileNotFoundError: [Errno 2] No such file or directory: '../data_jpar/20250209_joined_events_members.csv'

In [ ]:
##### ADDED BY JACOB -- GRABBING SOME DATA FOR DASHBOARD
# read in event information (downloaded from google sheets)
event_information_df = pd.read_csv('../dashboard/data/Cumulative JPAR Event Data - Event Information.csv')

# merge some columns onto our dataframe
cols_to_keep = ["Event ID", "Event Name", "Date"]  # whatever you want
joined = joined_events_members_filtered.merge(
    event_information_df[cols_to_keep],
    how="left",
    left_on="event_id",
    right_on="Event ID"
)
# rename 'joined' back to jointed_events_members_filtered 
# for consistency with the rest of the code
joined_events_members_filtered = joined.copy()

# change the columns im including to get more information
calculation_df = (
    joined_events_members_filtered[
        joined_events_members_filtered['membership_status'].isin(['ACTIVE', 'EXPIRED_RECENT'])
    ]
    .copy()
)

In [ ]:
# --- Ensure sorting and correct dtypes ---
calculation_df['event_id'] = calculation_df['event_id'].astype(float)
calculation_df = calculation_df.sort_values(['event_id', 'full_name'])

# --- Add event piece count ---
calculation_df['piece_count'] = calculation_df.groupby('event_id')['pieces_assembled'].transform('max')

# --- Compute completion time (seconds) ---
calculation_df['completion_seconds'] = (1 / calculation_df['ppm']) * calculation_df['piece_count'] * 60

# --- Event-level stats ---
event_avg = calculation_df.groupby('event_id')['completion_seconds'].transform('mean')
event_std = calculation_df.groupby('event_id')['completion_seconds'].transform('std')

calculation_df['adj_ind_JPAR'] = calculation_df['completion_seconds'] / event_avg
calculation_df['adj_ind_zscore_JPAR'] = (calculation_df['completion_seconds'] - event_avg) / event_std

# --- Initialize tracking columns ---
calculation_df['previous_JPAR'] = np.nan
calculation_df['JPAR'] = np.nan

# --- Track rolling JPAR per person ---
previous_jpars = {}

for idx, row in calculation_df.iterrows():
    name = row['full_name']
    current_jpar = row['adj_ind_JPAR']

    if name not in previous_jpars:
        # First event for this person
        prev_jpar = np.nan
        jpar = current_jpar
        previous_jpars[name] = [current_jpar]
    else:
        # Average across *all previous* JPARs
        prev_jpar = np.mean(previous_jpars[name])

        # Current JPAR could be the cumulative average including this event
        jpar = np.mean(previous_jpars[name] + [current_jpar])

        # Append this event’s JPAR to the history
        previous_jpars[name].append(current_jpar)

    calculation_df.at[idx, 'previous_JPAR'] = prev_jpar
    calculation_df.at[idx, 'JPAR'] = jpar

In [ ]:
##### ADDED BY JACOB ---- FORMATTING FOR EASE OF COMPATABILITY WITH EXISTING CODE I HAVE
# create a new dataframe to match the expected puzzleboard format
output_df = calculation_df.copy()
#### first i need to create a "Rank" for each person in each event;
#### this is just their place in the event. doing this by sorting
#### by completion_seconds, which is the extrapolated time
output_df["Rank"] = (
    output_df.groupby("event_id")["completion_seconds"]
      .rank(method="min", ascending=True)
)
# note that there are a few na's here, but ignoring them for now
###### FIX THIS 
###### FIX THIS 
###### FIX THIS 
###### FIX THIS 
output_df = output_df[output_df["Rank"].notna()]
# add a name column for clean names
output_df["Name"] = output_df["first_name"] + " " + output_df["last_name"]

###### column for Time in HH:MM:SS format,  but have to clean a bit
# make sure completion_seconds is numeric
output_df["completion_seconds"] = pd.to_numeric(output_df["completion_seconds"], errors="coerce")

# find max seconds for each event
output_df["event_max_seconds"] = output_df.groupby("event_id")["completion_seconds"].transform("max")

# function to convert seconds to HH:MM:SS
def seconds_to_hms(sec):
    h = int(sec // 3600)
    m = int((sec % 3600) // 60)
    s = int(sec % 60)
    return f"{h:02d}:{m:02d}:{s:02d}"

# new column 'Time':
# - if not DNF → format actual completion_seconds
# - if DNF     → format event_max_seconds
output_df["Time"] = output_df.apply(
    lambda row: seconds_to_hms(row["completion_seconds"])
    if row["completion_time"] != "DNF"
    else seconds_to_hms(row["event_max_seconds"]),
    axis=1,
)

# add pieces remaining
output_df['Remaining'] = output_df['piece_count']-output_df['pieces_assembled']

# change Date column format
output_df["Date"] = pd.to_datetime(output_df["Date"], format="mixed").dt.strftime("%Y-%m-%d")

# Pieces column for piece_count
output_df['Pieces'] = output_df['piece_count']

# Event column represent the event id for now i suppose
output_df['Event'] = output_df['event_id']

# Full_Event for event name
output_df['Full_Event'] = output_df['Event Name']

# time_in_seconds 
####### NEEDS TO BE CORRECTED
####### NEEDS TO BE CORRECTED
####### NEEDS TO BE CORRECTED
####### NEEDS TO BE CORRECTED
# has different puzzleboard meaning
output_df['time_in_seconds'] = output_df['completion_seconds']

# PTR In = prev_jpar
output_df['PTR In'] = output_df['previous_JPAR']

# PTR Out = JPAR
output_df['PTR Out'] = output_df['JPAR']

# PPM
output_df['PPM'] = output_df['ppm'] 

# corrected_time
output_df['corrected_time'] = output_df['completion_seconds']

# Total Events
output_df["Total Events"] = output_df.groupby("member_id")["event_id"].transform("count")

# rename PTR to JPAR
output_df = output_df.rename(columns={'PTR In': 'JPAR In', 'PTR Out': 'JPAR Out'})

##### Find latest JPAR information
# Sort by member_id and Date so the most recent is last
output_df = output_df.sort_values(['member_id', 'Date'])

# Group by member_id and pick the last (most recent) row for each
latest = output_df.groupby('member_id').last().reset_index()

# Rename the merged columns
latest = latest.rename(columns={
    'JPAR Out': 'Latest JPAR',
    'Full_Event': 'Latest Event',
    'Date': 'Latest Event Date'
})


# Create the new columns in the original dataframe
output_df = output_df.merge(
    latest[['member_id', 'Latest JPAR', 'Latest Event', 'Latest Event Date']],
    on='member_id',
    how='left'
)

In [ ]:
# relevant columns
output_df = output_df[['Rank','Name','Time','Remaining','Date','Full_Event',
                       'Latest JPAR', 'Latest Event', 'Latest Event Date',
                       'Pieces','Event','time_in_seconds','JPAR In','member_id',
                       'JPAR Out','PPM','corrected_time','Total Events']]

In [10]:
output_df

,Rank,Name,Time,Remaining,Date,Full_Event,Latest JPAR,Latest Event,Latest Event Date,Pieces,Event,time_in_seconds,JPAR In,member_id,JPAR Out,PPM,corrected_time,Total Events
0,1.0,Tammy McLeod,00:45:11,0.0,2023-09-21,WJPC 2023 First Round D,0.862172,Saint Paul Winter Carnival Solo,2025-01-26,500.0,230921.04,2712.0,NaN,17593433.0,0.693900,11.061947,2712.0,8
1,4.0,Tammy McLeod,00:53:38,0.0,2023-09-22,WJPC 2023 Semifinal 2,0.862172,Saint Paul Winter Carnival Solo,2025-01-26,500.0,230922.02,3218.0,0.693900,17593433.0,0.767208,9.322561,3218.0,8
2,14.0,Tammy McLeod,00:59:09,0.0,2023-09-23,WJPC 2023 Final,0.862172,Saint Paul Winter Carnival Solo,2025-01-26,500.0,230923.00,3550.0,0.767208,17593433.0,0.830374,8.450704,3550.0,8
3,4.0,Tammy McLeod,00:47:04,0.0,2024-09-19,WJPC 2024 First Round D,0.862172,Saint Paul Winter Carnival Solo,2025-01-26,500.0,240919.01,2824.0,0.830374,17593433.0,0.823650,10.623229,2824.0,8
4,9.0,Tammy McLeod,00:48:26,0.0,2024-09-20,WJPC 2024 Semifinal 2,0.862172,Saint Paul Winter Carnival Solo,2025-01-26,500.0,240920.02,2906.0,0.823650,17593433.0,0.861196,10.323469,2906.0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2462,134.0,Stephanie Klenzing,01:16:12,0.0,2025-02-08,Speed Puzzling # 292,1.212010,Speed Puzzling # 292,2025-02-08,500.0,250208.00,4572.0,1.071919,24098367.0,1.212010,6.561680,4572.0,2
2463,16.0,Christie Dizzia,02:42:03,0.0,2025-01-10,FL2025 - Florida Speed Puzzling Championship,1.543276,FL2025 - Florida Speed Puzzling Championship,2025-01-10,500.0,250110.01,9723.0,NaN,24099749.0,1.543276,3.085467,9723.0,1
2464,27.0,Jennifer McGinnis,01:34:29,0.0,2025-02-01,TX2025 - Texas Speed Puzzling Championship,0.967167,TX2025 - Texas Speed Puzzling Championship,2025-02-01,500.0,250201.00,5669.0,NaN,24108256.0,0.967167,5.291939,5669.0,1
2465,6.0,Melinda Shokler,01:03:14,0.0,2025-02-01,TX2025 - Texas Speed Puzzling Championship,0.647280,TX2025 - Texas Speed Puzzling Championship,2025-02-01,500.0,250201.00,3794.0,NaN,24108422.0,0.647280,7.907222,3794.0,1


In [25]:
output_df.to_pickle('./data/test_pickle.pkl')

In [3]:
!ls

README.md                  notes.txt
data                       requirements.txt
jpar_recalc_notebook.ipynb streamlit_app.py
nav                        utils


# Puzzleboard Format Comparison

In [9]:
data = pd.read_pickle('../dashboard/data/250811_scrape.pkl')
# get only the 500 pieces and those with data entry error
data = data[(data['Pieces'] == 500) | (data['Pieces'].isna())]

In [39]:
print(data)

       Rank                      Name     Time  \
0       348     Alfredo Sánchez Gómez  2:30:00   
1       303     Alba Erustes González  2:30:00   
2       290    Beatriz Luengo Tejedor  2:30:00   
3       352    Alba Sánchez Fernández  2:30:00   
4       304  Alba Hernández Hernández  2:30:00   
...     ...                       ...      ...   
32227   152              Toni Fussell  2:30:00   
32228   153            Alannah Gillis  2:30:00   
32229   154            Maegen Purcell  2:30:00   
32230   155             Michelle Chan  2:30:00   
32231   156      Sally Garmony-Burton  2:30:00   

                                        Remaining       Date  Pieces  \
0      201.00000000000000000000000000000000000000 2024-06-15     500   
1       47.00000000000000000000000000000000000000 2024-06-15     500   
2        5.00000000000000000000000000000000000000 2024-06-15     500   
3      240.00000000000000000000000000000000000000 2024-06-15     500   
4       50.0000000000000000000000000000

In [11]:
data.columns

Index(['Rank', 'Name', 'Time', 'Remaining', 'Date', 'Pieces', 'Event',
       'Full_Event', 'time_in_seconds', 'PTR In', 'PTR Out',
       'Avg PTR In (Event)', '12-Month Avg Completion Time', 'PPM',
       'Latest JPAR Out', 'time_penalty', 'corrected_time', 'Total Events'],
      dtype='object')

In [38]:
calculation_df.columns

Index(['full_name', 'member_id', 'completion_time', 'event_id',
       'pieces_assembled', 'max_time_seconds', 'ppm', 'first_name',
       'last_name', 'mp_id_deduped', 'membership_cutoff_date',
       'membership_status', 'mp_id', 'initiated_date', 'match_type',
       'Event ID', 'Event Name', 'Date', 'piece_count', 'completion_seconds',
       'adj_ind_JPAR', 'adj_ind_zscore_JPAR', 'previous_JPAR', 'JPAR'],
      dtype='object')

# Trying to implement my own JPAR

I wanted to try to implement my own JPAR calculation following the Excel sheet to make sure I know what's going on under the hood.


- It seems like this perfectly reproduces the Excel sheets, with the caveat that there is some weird rounding at the ~4th or 5th decimal due to how Excel treats numbers/time calculations. Other than that, this is perfect!


In [3]:
# Import required packages
import pandas as pd
import numpy as np
import re
import os
from datetime import datetime

In [4]:
# Read in the data
joined_events_members_filtered = pd.read_csv('/Users/jacobpilawa/Desktop/puzzle_data/usajpa_work/scratchwork/data_jpar/20250209_joined_events_members.csv')
jpar_times = pd.read_csv('/Users/jacobpilawa/Desktop/puzzle_data/usajpa_work/scratchwork/data_jpar/jpar_times.csv')

# remove ppl who joined after the cuttoff
joined_events_members_filtered = joined_events_members_filtered[
    pd.to_datetime(joined_events_members_filtered['initiated_date'], errors='coerce') < pd.Timestamp('2025-02-09')
]

# and exclude connor
# Drop row where full_name == 'conner delaat'
joined_events_members_filtered = joined_events_members_filtered[joined_events_members_filtered['full_name']!= 'conner delaat']


In [5]:
##### ADDED BY JACOB -- GRABBING SOME DATA FOR DASHBOARD
# read in event information (downloaded from google sheets)
event_information_df = pd.read_csv('../dashboard/data/Cumulative JPAR Event Data - Event Information.csv')

# merge some columns onto our dataframe
cols_to_keep = ["Event ID", "Event Name", "Date"]  # whatever you want
joined = joined_events_members_filtered.merge(
    event_information_df[cols_to_keep],
    how="left",
    left_on="event_id",
    right_on="Event ID"
)
# rename 'joined' back to jointed_events_members_filtered 
# for consistency with the rest of the code
joined_events_members_filtered = joined.copy()

# change the columns im including to get more information
calculation_df = (
    joined_events_members_filtered[
        joined_events_members_filtered['membership_status'].isin(['ACTIVE', 'EXPIRED_RECENT'])
    ]
    .copy()
)

In [6]:
# --- Ensure sorting and correct dtypes ---
calculation_df['event_id'] = calculation_df['event_id'].astype(float)
calculation_df = calculation_df.sort_values(['event_id', 'full_name'])

# --- Add event piece count ---
calculation_df['piece_count'] = calculation_df.groupby('event_id')['pieces_assembled'].transform('max')

# --- Compute completion time (seconds) ---
calculation_df['completion_seconds'] = (1 / calculation_df['ppm']) * calculation_df['piece_count'] * 60



In [7]:
import pandas as pd
import numpy as np

def compute_jpar(df_in):
    """
    Compute JPAR for every row and add:
      - Event_Mean_Time, Event_JPAR,
      - Expected_Event_Average, MEAN_Expected_Event_Average,
      - Adjusted_Event_JPAR,
      - Prev_JPAR, JPAR_Out,
      - Latest_JPAR
    Returns a new dataframe (copy).
    """
    df = df_in.copy()

    # Round completion_seconds to nearest second
    #df["completion_seconds"] = df["completion_seconds"].round().astype("Int64")
    df["completion_seconds"] = df["completion_seconds"].astype("Int64")

    # Ensure Date is datetime
    if "Date" in df.columns:
        df["Date"] = pd.to_datetime(df["Date"])
    else:
        df["Date"] = pd.to_datetime("1970-01-01")

    # Sort for chronological order
    df = df.sort_values(by=["Date", "event_id"]).copy()

    # Initialize result columns
    cols = [
        "Event_Mean_Time", "Event_JPAR", "Expected_Event_Average",
        "MEAN_Expected_Event_Average", "Adjusted_Event_JPAR",
        "Prev_JPAR", "JPAR_Out", "Latest_JPAR"
    ]
    for c in cols:
        df[c] = np.nan

    latest_jpar_running = {}

    # Loop over events
    events = df[["Date", "event_id"]].drop_duplicates().sort_values(["Date", "event_id"])
    for _, ev in events.iterrows():
        mask = (df["Date"] == ev["Date"]) & (df["event_id"] == ev["event_id"])
        sub_idx = df.index[mask]
        sub = df.loc[sub_idx]

        # Round mean time
        mean_time = sub.loc[sub["completion_seconds"].notna(), "completion_seconds"].mean()
        mean_time = np.round(mean_time) if not np.isnan(mean_time) else np.nan
        df.loc[sub_idx, "Event_Mean_Time"] = mean_time

        # Event JPAR (time / mean_time)
        df.loc[sub_idx, "Event_JPAR"] = sub["completion_seconds"] / mean_time if not np.isnan(mean_time) else np.nan

        # Expected Event Average
        expected_vals = []
        for idx in sub_idx:
            member = df.at[idx, "member_id"]
            prev = latest_jpar_running.get(member)
            df.at[idx, "Prev_JPAR"] = prev if prev is not None else np.nan

            if prev is not None and not pd.isna(prev):
                expected = df.at[idx, "completion_seconds"] * (1/prev)
                #expected = np.round(expected)
                expected = expected
                df.at[idx, "Expected_Event_Average"] = expected
                expected_vals.append(expected)

        # Round mean expected
        mean_expected = np.mean(expected_vals) if expected_vals else np.nan
        #mean_expected = np.round(mean_expected) if not np.isnan(mean_expected) else np.nan
        mean_expected = mean_expected if not np.isnan(mean_expected) else np.nan
        df.loc[sub_idx, "MEAN_Expected_Event_Average"] = mean_expected

        # Adjusted_Event_JPAR
        if not np.isnan(mean_expected):
            df.loc[sub_idx, "Adjusted_Event_JPAR"] = df.loc[sub_idx, "completion_seconds"] / mean_expected
        else:
            df.loc[sub_idx, "Adjusted_Event_JPAR"] = df.loc[sub_idx, "Event_JPAR"]

        # Update JPAR_Out
        for idx in sub_idx:
            member = df.at[idx, "member_id"]
            prev = latest_jpar_running.get(member)
            adj = df.at[idx, "Adjusted_Event_JPAR"]

            if prev is None or pd.isna(prev):
                jpar_out = adj
            else:
                jpar_out = (prev + adj) / 2.0

            df.at[idx, "JPAR_Out"] = jpar_out
            latest_jpar_running[member] = jpar_out

    # Final Latest_JPAR
    final_jpar_series = (
        df.sort_values(["Date", "event_id"])
          .groupby("member_id", sort=False)["JPAR_Out"]
          .last()
    )
    df["Latest_JPAR"] = df["member_id"].map(final_jpar_series)

    return df


In [29]:
import pandas as pd

# ----------------------------
# CONSISTENT PERFORMANCE
# ----------------------------
test_data1 = [
    {"event_id": 1, "Date": "2024-01-01", "name": "Road Runner", "completion_time": "0:22:50", "completion_seconds": 1370},
    {"event_id": 1, "Date": "2024-01-01", "name": "Sonic the Hedgehog", "completion_time": "0:29:35", "completion_seconds": 1775},
    {"event_id": 1, "Date": "2024-01-01", "name": "Princess Peach", "completion_time": "0:40:12", "completion_seconds": 2412},
    {"event_id": 1, "Date": "2024-01-01", "name": "Koopa Troopa", "completion_time": "0:43:39", "completion_seconds": 2619},
    {"event_id": 1, "Date": "2024-01-01", "name": "Slowpoke", "completion_time": "0:56:26", "completion_seconds": 3386},

    {"event_id": 2, "Date": "2024-02-01", "name": "Princess Peach", "completion_time": "0:40:12", "completion_seconds": 2412},
    {"event_id": 2, "Date": "2024-02-01", "name": "Koopa Troopa", "completion_time": "0:43:39", "completion_seconds": 2619},
    {"event_id": 2, "Date": "2024-02-01", "name": "Slowpoke", "completion_time": "0:56:26", "completion_seconds": 3386},

    {"event_id": 3, "Date": "2024-03-01", "name": "Road Runner", "completion_time": "0:22:50", "completion_seconds": 1370},
    {"event_id": 3, "Date": "2024-03-01", "name": "Sonic the Hedgehog", "completion_time": "0:29:35", "completion_seconds": 1775},
    {"event_id": 3, "Date": "2024-03-01", "name": "Princess Peach", "completion_time": "0:40:12", "completion_seconds": 2412},
]

# ----------------------------
# PERFORMANCE VS. PLACEMENT
# ----------------------------
test_data2 = [
    {"event_id": 1, "Date": "2024-01-01", "name": "Road Runner", "completion_time": "0:22:50", "completion_seconds": 1370},
    {"event_id": 1, "Date": "2024-01-01", "name": "Sonic the Hedgehog", "completion_time": "0:29:35", "completion_seconds": 1775},
    {"event_id": 1, "Date": "2024-01-01", "name": "Princess Peach", "completion_time": "0:40:12", "completion_seconds": 2412},
    {"event_id": 1, "Date": "2024-01-01", "name": "Koopa Troopa", "completion_time": "0:43:39", "completion_seconds": 2619},
    {"event_id": 1, "Date": "2024-01-01", "name": "Slowpoke", "completion_time": "0:56:26", "completion_seconds": 3386},

    {"event_id": 2, "Date": "2024-02-01", "name": "Princess Peach", "completion_time": "0:43:00", "completion_seconds": 2580},
    {"event_id": 2, "Date": "2024-02-01", "name": "Koopa Troopa", "completion_time": "0:43:39", "completion_seconds": 2619},
    {"event_id": 2, "Date": "2024-02-01", "name": "Slowpoke", "completion_time": "0:56:26", "completion_seconds": 3386},

    {"event_id": 3, "Date": "2024-03-01", "name": "Road Runner", "completion_time": "0:22:50", "completion_seconds": 1370},
    {"event_id": 3, "Date": "2024-03-01", "name": "Sonic the Hedgehog", "completion_time": "0:29:35", "completion_seconds": 1775},
    {"event_id": 3, "Date": "2024-03-01", "name": "Princess Peach", "completion_time": "0:33:01", "completion_seconds": 1981},
]

# ----------------------------
# HIGH PERFORMANCE W UNKNOWN PUZZLERS
# ----------------------------
test_data3 = [
    {"event_id": 1, "Date": "2024-01-01", "name": "Road Runner", "completion_time": "0:22:50", "completion_seconds": 1370},
    {"event_id": 1, "Date": "2024-01-01", "name": "Sonic the Hedgehog", "completion_time": "0:29:35", "completion_seconds": 1775},
    {"event_id": 1, "Date": "2024-01-01", "name": "Princess Peach", "completion_time": "0:40:12", "completion_seconds": 2412},
    {"event_id": 1, "Date": "2024-01-01", "name": "Koopa Troopa", "completion_time": "0:43:39", "completion_seconds": 2619},
    {"event_id": 1, "Date": "2024-01-01", "name": "Slowpoke", "completion_time": "0:56:26", "completion_seconds": 3386},

    {"event_id": 2, "Date": "2024-02-01", "name": "Princess Peach", "completion_time": "0:43:00", "completion_seconds": 2580},
    {"event_id": 2, "Date": "2024-02-01", "name": "Koopa Troopa", "completion_time": "0:43:39", "completion_seconds": 2619},
    {"event_id": 2, "Date": "2024-02-01", "name": "Slowpoke", "completion_time": "0:56:26", "completion_seconds": 3386},

    {"event_id": 3, "Date": "2024-03-01", "name": "Road Runner", "completion_time": "0:22:50", "completion_seconds": 1370},
    {"event_id": 3, "Date": "2024-03-01", "name": "Animal", "completion_time": "0:23:00", "completion_seconds": 1380},
    {"event_id": 3, "Date": "2024-03-01", "name": "Tasmanian Devil", "completion_time": "0:24:00", "completion_seconds": 1440},
    {"event_id": 3, "Date": "2024-03-01", "name": "Flash", "completion_time": "0:25:00", "completion_seconds": 1500},
    {"event_id": 3, "Date": "2024-03-01", "name": "Sonic the Hedgehog", "completion_time": "0:29:35", "completion_seconds": 1775},
    {"event_id": 3, "Date": "2024-03-01", "name": "Princess Peach", "completion_time": "0:33:01", "completion_seconds": 1981},
]

# ----------------------------
# TOUGH PUZZLE LONGER TIME
# ----------------------------
test_data4 = [
    {"event_id": 1, "Date": "2024-01-01", "name": "Road Runner", "completion_time": "0:22:50", "completion_seconds": 1370},
    {"event_id": 1, "Date": "2024-01-01", "name": "Sonic the Hedgehog", "completion_time": "0:29:35", "completion_seconds": 1775},
    {"event_id": 1, "Date": "2024-01-01", "name": "Princess Peach", "completion_time": "0:40:12", "completion_seconds": 2412},
    {"event_id": 1, "Date": "2024-01-01", "name": "Koopa Troopa", "completion_time": "0:43:39", "completion_seconds": 2619},
    {"event_id": 1, "Date": "2024-01-01", "name": "Slowpoke", "completion_time": "0:56:26", "completion_seconds": 3386},

    {"event_id": 2, "Date": "2024-02-01", "name": "Princess Peach", "completion_time": "0:43:00", "completion_seconds": 2580},
    {"event_id": 2, "Date": "2024-02-01", "name": "Koopa Troopa", "completion_time": "0:43:39", "completion_seconds": 2619},
    {"event_id": 2, "Date": "2024-02-01", "name": "Slowpoke", "completion_time": "0:56:26", "completion_seconds": 3386},

    {"event_id": 3, "Date": "2024-03-01", "name": "Road Runner", "completion_time": "0:32:50", "completion_seconds": 1970},
    {"event_id": 3, "Date": "2024-03-01", "name": "Animal", "completion_time": "0:33:00", "completion_seconds": 1980},
    {"event_id": 3, "Date": "2024-03-01", "name": "Tasmanian Devil", "completion_time": "0:34:00", "completion_seconds": 2040},
    {"event_id": 3, "Date": "2024-03-01", "name": "Flash", "completion_time": "0:35:00", "completion_seconds": 2100},
    {"event_id": 3, "Date": "2024-03-01", "name": "Sonic the Hedgehog", "completion_time": "0:39:35", "completion_seconds": 2375},
    {"event_id": 3, "Date": "2024-03-01", "name": "Princess Peach", "completion_time": "0:50:12", "completion_seconds": 3012},
]


In [30]:
test_df_A = pd.DataFrame(test_data1)
test_df_B = pd.DataFrame(test_data2)
test_df_C = pd.DataFrame(test_data3)
test_df_D = pd.DataFrame(test_data4)

dfs = [test_df_A, test_df_B, test_df_C, test_df_D]

# Add member_id to each DataFrame
for df in dfs:
    df["member_id"] = df.groupby("name").ngroup() + 1

In [31]:
jpars = [compute_jpar(df) for df in dfs]

In [36]:
labels = ['Consistent Performance',
          'Performance vs. Placement',
          'High Performance w Unknown Puzzlers',
          'Tough Puzzle Longer Time']

In [41]:
for jpar, title in zip(jpars, labels):
    print(f""" '{title}' """ + ' Results:')
    latest_jpar = (
        jpars[0].sort_values(["member_id", "Date"])
          .groupby("member_id")
          .tail(1)[["name", "Latest_JPAR"]]
          .reset_index(drop=True)
    )
    
    print(latest_jpar)
    print('\n')


 'Consistent Performance'  Results:
                 name  Latest_JPAR
0        Koopa Troopa     1.132785
1      Princess Peach     1.043253
2         Road Runner     0.592561
3            Slowpoke     1.464533
4  Sonic the Hedgehog     0.767734


 'Performance vs. Placement'  Results:
                 name  Latest_JPAR
0        Koopa Troopa     1.132785
1      Princess Peach     1.043253
2         Road Runner     0.592561
3            Slowpoke     1.464533
4  Sonic the Hedgehog     0.767734


 'High Performance w Unknown Puzzlers'  Results:
                 name  Latest_JPAR
0        Koopa Troopa     1.132785
1      Princess Peach     1.043253
2         Road Runner     0.592561
3            Slowpoke     1.464533
4  Sonic the Hedgehog     0.767734


 'Tough Puzzle Longer Time'  Results:
                 name  Latest_JPAR
0        Koopa Troopa     1.132785
1      Princess Peach     1.043253
2         Road Runner     0.592561
3            Slowpoke     1.464533
4  Sonic the Hedgehog     0

In [21]:
jpars[1]

,event_id,Date,name,completion_time,completion_seconds,member_id,Event_Mean_Time,Event_JPAR,Expected_Event_Average,MEAN_Expected_Event_Average,Adjusted_Event_JPAR,Prev_JPAR,JPAR_Out,Latest_JPAR
0,1,2024-01-01,Road Runner,0:22:50,1370,3,2312.0,0.592561,NaN,NaN,0.592561,NaN,0.592561,0.613410
1,1,2024-01-01,Sonic the Hedgehog,0:29:35,1775,5,2312.0,0.767734,NaN,NaN,0.767734,NaN,0.767734,0.794746
2,1,2024-01-01,Princess Peach,0:40:12,2412,2,2312.0,1.043253,NaN,NaN,1.043253,NaN,1.043253,0.991990
3,1,2024-01-01,Koopa Troopa,0:43:39,2619,1,2312.0,1.132785,NaN,NaN,1.132785,NaN,1.132785,1.119858
4,1,2024-01-01,Slowpoke,0:56:26,3386,4,2312.0,1.464533,NaN,NaN,1.464533,NaN,1.464533,1.447820
5,2,2024-02-01,Princess Peach,0:43:00,2580,2,2862.0,0.901468,2473.0,2366.0,1.090448,1.043253,1.066850,0.991990
6,2,2024-02-01,Koopa Troopa,0:43:39,2619,1,2862.0,0.915094,2312.0,2366.0,1.106932,1.132785,1.119858,1.119858
7,2,2024-02-01,Slowpoke,0:56:26,3386,4,2862.0,1.183089,2312.0,2366.0,1.431107,1.464533,1.447820,1.447820
8,3,2024-03-01,Road Runner,0:22:50,1370,3,1709.0,0.801638,2312.0,2160.0,0.634259,0.592561,0.613410,0.613410
9,3,2024-03-01,Sonic the Hedgehog,0:29:35,1775,5,1709.0,1.038619,2312.0,2160.0,0.821759,0.767734,0.794746,0.794746


In [ ]:
# performance vs placement for example:

# event 2 has Koopa Troopa's "Adjusted Event JPAR" = 1.10708
# but the formula is C16/E25 = 2619/2366= 1.10693
# however if you compute the cell E25 without rounding, you would get 2365.66666667
# which implies Adjusted Event Jpar = 1.10708 which is in fact what I'm finding

# for the new jpar, the excel sheet calculates this as mean(E10,F16) = mean(1.132785467,1.107082069) = 1.119933768
# however i would say that this is mean(E10,F16) = mean(1.10708750176,1.132785467) = 1.11993648438

In [23]:
(1.10708750176+1.132785467)/2

1.11993648438

In [52]:
int(2312.6)

2312

In [50]:
jpar

,event_id,Date,name,completion_time,completion_seconds,member_id,Event_Mean_Time,Event_JPAR,Expected_Event_Average,MEAN_Expected_Event_Average,Adjusted_Event_JPAR,Prev_JPAR,JPAR_Out,Latest_JPAR
0,1,2024-01-01,Road Runner,0:22:50,1370,3,2312.400000,0.592458,NaN,NaN,0.592458,NaN,0.592458,0.777722
1,1,2024-01-01,Sonic the Hedgehog,0:29:35,1775,5,2312.400000,0.767601,NaN,NaN,0.767601,NaN,0.767601,1.007633
2,1,2024-01-01,Princess Peach,0:40:12,2412,2,2312.400000,1.043072,NaN,NaN,1.043072,NaN,1.043072,1.281750
3,1,2024-01-01,Koopa Troopa,0:43:39,2619,1,2312.400000,1.132590,NaN,NaN,1.132590,NaN,1.132590,0.942581
4,1,2024-01-01,Slowpoke,0:56:26,3386,4,2312.400000,1.464280,NaN,NaN,1.464280,NaN,1.464280,1.218625
5,2,2024-02-01,Princess Peach,0:40:12,2412,2,2805.666667,0.859689,2515.889984,3480.064147,0.693091,1.043072,0.868081,1.281750
6,2,2024-02-01,Koopa Troopa,0:43:39,2619,1,2805.666667,0.933468,2966.251946,3480.064147,0.752572,1.132590,0.942581,0.942581
7,2,2024-02-01,Slowpoke,0:56:26,3386,4,2805.666667,1.206843,4958.050510,3480.064147,0.972971,1.464280,1.218625,1.218625
8,3,2024-03-01,Road Runner,0:22:50,1370,3,1852.333333,0.739608,811.667532,1422.657075,0.962987,0.592458,0.777722,0.777722
9,3,2024-03-01,Sonic the Hedgehog,0:29:35,1775,5,1852.333333,0.958251,1362.491351,1422.657075,1.247665,0.767601,1.007633,1.007633


In [39]:
jpar[['Latest_JPAR','name']]

,Latest_JPAR,name
0,0.777722,Road Runner
1,1.007633,Sonic the Hedgehog
2,1.281750,Princess Peach
3,0.942581,Koopa Troopa
4,1.218625,Slowpoke
5,1.281750,Princess Peach
6,0.942581,Koopa Troopa
7,1.218625,Slowpoke
8,0.777722,Road Runner
9,1.007633,Sonic the Hedgehog


In [12]:
df_jpar = compute_jpar(calculation_df)


In [18]:
# Take the most recent JPAR_Out for each person
final_jpar = (
    df_jpar.sort_values(["Date", "event_id"])  # chronological
          .groupby("member_id")
          .tail(1)[["member_id", "full_name", "JPAR_Out"]]
)

# Sort by JPAR (lower is better)
final_jpar = final_jpar.sort_values("JPAR_Out").reset_index(drop=True)

In [20]:
final_jpar.head(30)

,member_id,full_name,JPAR_Out
0,21836692.0,jenn bodshaug,0.468520
1,22402963.0,nancy siner,0.480014
2,21428556.0,caryn knill,0.492840
3,22814742.0,grayson shannon,0.523304
4,23970677.0,alexandra wright,0.535468
5,23765860.0,kate yee,0.601181
6,20859985.0,jessa douglas,0.602483
7,22546380.0,hannah wensing,0.603282
8,23742868.0,katie erickson,0.618285
9,17731921.0,kaitlin mackenzie,0.620393
